In [ ]:
import re
import pandas as pd
from collections import namedtuple
import requests
from bs4 import BeautifulSoup
import time
from urllib.request import urlopen
import urllib.request, urllib.parse, urllib.error
import random

UserReview = namedtuple('UserReview', ['url', 'review_title', 'comment', 'rating', 'date', 'username', 'profile_url', 'verified_purchase'])

class AmazonScraper:
#     review_date_pattern = re.compile('(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?) \d+, \d{4}')
    product_name_pattern = re.compile('^https:\/{2}www.amazon.co.uk\/(.+)\/product-reviews')
    def __init__(self):
        # create a browser session
        #change user agent if got block
        self.session = requests.Session()
        self.session.headers['User-Agent'] = 'My User Agent xxxdd'
        self.session.headers['Accept'] = 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'
        self.session.headers['Accept-Language'] = 'en-US,en;q=0.5'
        self.session.headers['Connection'] = 'keep-alive'
        self.session.headers['Upgrade-Insecure-Requests'] = '1'

    def scrapeReviews(self, url, page_num, filter_by='recent'):
        """
        args
            filter_by: recent or helpful
        return
            namedtuple
        """
        try:
            url1 = url.replace('dp', 'product-reviews')+'&reviewerType=all_reviews'
            review_url = re.search('^.+(?=\/)', url1).group()
            review_url = review_url + '/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy={0}&pageNumber={1}'.format(filter_by, page_num)
#             print('Processing {0}...'.format(review_url))
            response = self.session.get(review_url)

            product_name = self.product_name_pattern.search(url1).group(1) if self.product_name_pattern.search(url1) else ''
            if not product_name:
                print('url is invalid. Please check the url.')
                return
            else:
                product_name = product_name.replace('-', ' ')

            soup = BeautifulSoup(response.content, 'html.parser')
            review_list = soup.find('div', {'id': 'cm_cr-review_list' })    
            
            if not review_list:
                return
            
            reviews = []
            product_reviews = review_list.find_all('div', {'data-hook': 'review'}) # return reviews
            
            if not product_reviews:
                return
            
            for product_review in product_reviews:
                if (product_review.find('a', {'data-hook': 'review-title'})) is None:
#                     print('Include other country')
                    break
                else:
                    review_title = product_review.find('a', {'data-hook': 'review-title'}).text.strip()
                    verified_purchase = True if product_review.find('span', {'data-hook': 'avp-badge'}) else False
                    review_body = product_review.find('span', {'data-hook': 'review-body'}).text.strip()
                    rating = product_review.find('i', {'data-hook': 'review-star-rating'}).text
    #                 review_date = self.review_date_pattern.search(product_review.find('span', {'data-hook': 'review-date'}).text).group(0)
                    review_date = product_review.find('span', {'data-hook': 'review-date'}).text
                    username = product_review.a.span.text
                    user_profile = 'https://amazon.co.uk/{0}'.format(product_review.a['href'])
                    reviews.append(UserReview(url, review_title, review_body, rating, review_date, username, user_profile, verified_purchase))
            return reviews
        except Exception as e:
            print(e)
            return None

In [4]:
user_agent_list = [
   #Chrome
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (Windows NT 5.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
    #Firefox
    'Mozilla/4.0 (compatible; MSIE 9.0; Windows NT 6.1)',
    'Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.1; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (Windows NT 6.2; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.0; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.3; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; WOW64; Trident/6.0)',
    'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; Trident/6.0)',
    'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0; .NET CLR 2.0.50727; .NET CLR 3.0.4506.2152; .NET CLR 3.5.30729)'
]

In [40]:

#Get Toal Pages

# Define the search query and URL
query = 'tommy hilfiger jewellery men'
page = 1
url = f'https://www.amazon.co.uk/s?k={query}&page={page}&ref=nb_sb_noss'
url = url.replace(" ", "%20")
# url = 'https://www.amazon.co.uk/s?k=tommy+hilfiger+jewellery+men&qid=1677873266&ref=sr_pg_1'
#Tissot url = https://www.amazon.co.uk/s?k=tissot+watches+for+men&crid=WR3VR7NUASFY&sprefix=tissot%2Caps%2C202&ref=nb_sb_ss_ts-doa-p_2_6
# session = requests.Session()
# session.headers['User-Agent'] = 'My User Agent 4.2'
# session.headers['Accept'] = 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'
# session.headers['Accept-Language'] = 'en-US,en;q=0.5'
# session.headers['Connection'] = 'keep-alive'
# session.headers['Upgrade-Insecure-Requests'] = '1'
# # Send a request to the search URL and get the HTML content of the page
# response = session.get(url)
# html_content = response.content
user_agent = random.choice(user_agent_list)
opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', user_agent)]
response = opener.open(url)
html_content = response.read()
soup = BeautifulSoup(html_content, 'html.parser')

result_text = soup.find('div', {'class': 'a-section a-spacing-small a-spacing-top-small'}).get_text().strip()
total_results = int(result_text.split()[2].replace(',', ''))

# results_per_page = len(soup.find_all('div', {'class': 's-result-item'}))
results_per_page = int(result_text.split()[0].replace(',', '').split('-')[1])
num_pages = total_results // results_per_page + (1 if total_results % results_per_page > 0 else 0)

print(f'Total number of results: {total_results}')
print(f'Results per page: {results_per_page}')
print(f'Number of pages: {num_pages}')


HTTPError: HTTP Error 503: Service Unavailable

In [16]:
###Only for one page
# query = 'tommy hilfiger jewellery women'
Title = []
Price = []
Rating = []
Description = []
url_key = []
df = pd.DataFrame(columns=['Name', 'Value', 'url'])
reviews = []
# session = requests.Session()
# session.headers['User-Agent'] = 'My User Agent 87.7'
# session.headers['Accept'] = 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'
# session.headers['Accept-Language'] = 'en-US,en;q=0.5'
# session.headers['Connection'] = 'keep-alive'
# session.headers['Upgrade-Insecure-Requests'] = '1'

#num_pages+1
for page in range(1,2):
#     url = f'https://www.amazon.co.uk/s?k={query}&page={page}&ref=nb_sb_noss'
#     # Parse the HTML content using BeautifulSoup
#     print(f'Page:{page}')
    
    url = 'https://www.amazon.co.uk/s?k=boss+jewellery+for+men&crid=33F2J8F5ZMF98&sprefix=boss+jewellery+for+men%2Caps%2C208&ref=nb_sb_noss_1' 
    user_agent = random.choice(user_agent_list)
    opener = urllib.request.build_opener()
    opener.addheaders = [('User-agent', user_agent)]
    response = opener.open(url)
    html_content = response.read()
#     response = session.get(url)
#     html_content = response.content
    
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Extract the URLs of the product pages from the search results
    product_urls = []
    results = soup.find_all('div', {'class': 's-result-item'})
    for result in results:
        link = result.find('a', {'class': 'a-link-normal s-no-outline'})

        if link:
            u = 'https://www.amazon.co.uk'+link.get('href')
            product_urls.append(u)
    # List

    # Scrape the product details for each product page
    for product_url in product_urls[:-1]:
        url_key.append(product_url)
#         response = session.get(product_url)
#         html_content = response.content
        response = opener.open(product_url)
        html_content = response.read()
        soup = BeautifulSoup(html_content, 'html.parser')
        title = soup.find('span', attrs={'id': 'productTitle'}).get_text().strip()
        Title.append(title)
        if not soup.find('span', attrs={'class': 'a-price-whole'}):
            Price.append(' ')
        else:
            price1 = soup.find('span', attrs={'class': 'a-price-whole'}).get_text().strip()
            price2 = soup.find('span', attrs={'class': 'a-price-fraction'}).get_text().strip()
            Price.append(price1+price2)
        if not soup.find('span', attrs={'class': 'a-icon-alt'}):
            Rating.append(' ')
        else:
            rating = soup.find('span', attrs={'class': 'a-icon-alt'}).get_text().strip()
            Rating.append(rating)
        if not soup.find('div', attrs={'id': 'productDescription'}):
            Description.append(' ')
        else:
            description = soup.find('div', attrs={'id': 'productDescription'}).get_text().strip()
            Description.append(description)
        
        #Detail page
        specifications_container = soup.find("table", attrs={"id": "technicalSpecifications_section_1"})
        if specifications_container:
            specification_rows = specifications_container.find_all("tr")

            # Iterate through each specification row and extract the relevant information
            for row in specification_rows:
                # Extract the specification name
                name = row.find("th").text.strip()

                # Extract the specification value
                value = row.find("td").text.strip()

            # Print the extracted information
                df = df.append({'Name': name, 'Value': value, 'url': product_url}, ignore_index=True)
        else:
            df.append({'Name': ' ', 'Value': ' ', 'url': product_url}, ignore_index=True)
            
        print('---------------------------------------------------------')
        
        print(f'Title: {title}')
        print(f'Price: {price1+price2}')
        print(f'Rating: {rating}')
        print(f'Description: {description}')
        
 
        amz_scraper = AmazonScraper()
        
#         product_url = product_url.replace('dp', 'product-reviews')+'&reviewerType=all_reviews'
        for page_num in range(1, 100):
            if amz_scraper.scrapeReviews(url=product_url, page_num=page_num) ==[] or amz_scraper.scrapeReviews(url=product_url, page_num=page_num) ==None:
#                 reviews.extend(UserReview(' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '))
                break
            reviews.extend(amz_scraper.scrapeReviews(url=product_url, page_num=page_num))
            time.sleep(5)

#         Reviews = pd.DataFrame(reviews)
        if product_url == product_urls[-2]:
            print(f'page {page} is finished')
            print('---------------------------------------------')
            print('\n')
        
        
        time.sleep(5)

---------------------------------------------------------
Title: BOSS Jewelry Men's Chain Link Collection Chain Necklace - 1580142, (logo may vary)
Price: 63.99
Rating: 4.4 out of 5 stars
Description: BOSS jewellery reflects the sophisticated character and craftsmanship for which all HUGO BOSS products are known. Synonymous with fashion, it stands out in the world of quality branded jewellerywith a style and presence of its own.


---------------------------------------------------------
Title: BOSS Jewelry Men's Chain Link Collection Chain Bracelet - 1580144M
Price: 50.15
Rating: 4.0 out of 5 stars
Description: BOSS jewellery reflects the sophisticated character and craftsmanship for which all HUGO BOSS products are known. Synonymous with fashion, it stands out in the world of quality branded jewellerywith a style and presence of its own.


---------------------------------------------------------
Title: BOSS Jewelry Men's SEAL Collection Bracelet Black - 1580047M
Price: 42.00
Rating: 4.3 out of 5 stars
Description: BOSS jewellery reflects the sophisticated character and craftsmanship for which all HUGO BOSS products are known. Synonymous with fashion, it stands out in the world of quality branded jewellerywith a style and presence of its own.


---------------------------------------------------------
Title: BOSS Jewelry Men's Chain For Him Collection Chain Necklace - 1580173
Price: 75.65
Rating: 4.2 out of 5 stars
Description: BOSS jewellery reflects the sophisticated character and craftsmanship for which all HUGO BOSS products are known. Synonymous with fashion, it stands out in the world of quality branded jewellerywith a style and presence of its own.


---------------------------------------------------------
Title: BOSS Jewelry Men's BRAIDED LEATHER Collection Bracelet Black - 1580152
Price: 42.00
Rating: 3.8 out of 5 stars
Description: BOSS jewellery reflects the sophisticated character and craftsmanship for which all HUGO BOSS products are known. Synonymous with fashion, it stands out in the world of quality branded jewellerywith a style and presence of its own.


...

---------------------------------------------------------
Title: BOSS Jewelry Men's Carter Collection Pendant Necklace - 1580315
Price: 99.00
Rating: 4.0 out of 5 stars
Description: Dress like a BOSS. BOSS jewelry reflects the dressing excellence codes and design for which all HUGO BOSS products are known. Synonymous with fashion, it stands out in the world of quality branded jewelry with a style and presence of its own. Tailored for the man and woman who draw attention in all situations. #BeYourOwnBOSS


---------------------------------------------------------
Title: BOSS Jewelry Men's Mixed Beads Collection Stone Bracelet - 1580272
Price: 54.07
Rating: 3.0 out of 5 stars
Description: Dress like a BOSS.BOSS jewelry reflects the dressing excellence codes and design for which all HUGO BOSS products are known. Synonymous with fashion, it stands out in the world of quality branded jewelry with a style and presence of its own. Tailored for the man and woman who draw attention in all situations.BeYourOwnBOSS


---------------------------------------------------------
Title: BOSS Jewelry Men's JACE Collection Leather Bracelet Black - 1580337M
Price: 58.65
Rating: 5.0 out of 5 stars
Description: Dress like a BOSS. BOSS jewelry reflects the dressing excellence codes and design for which all HUGO BOSS products are known. Synonymous with fashion, it stands out in the world of quality branded jewelry with a style and presence of its own. Tailored for the man and woman who draw attention in all situations. #BeYourOwnBOSS.


---------------------------------------------------------
Title: BOSS Jewelry Men's ID Collection Chain Bracelet - 1580051M
Price: 69.20
Rating: 3.8 out of 5 stars
Description: BOSS jewellery reflects the sophisticated character and craftsmanship for which all HUGO BOSS products are known. Synonymous with fashion, it stands out in the world of quality branded jewellerywith a style and presence of its own.


page 1 is finished
---------------------------------------------




In [42]:
query = 'tommy hilfiger jewellery men'
Title = []
Price = []
Rating = []
Description = []
url_key = []
df = pd.DataFrame(columns=['Name', 'Value', 'url'])
reviews = []
# session = requests.Session()
# session.headers['User-Agent'] = 'My User Agent 87.7'
# session.headers['Accept'] = 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'
# session.headers['Accept-Language'] = 'en-US,en;q=0.5'
# session.headers['Connection'] = 'keep-alive'
# session.headers['Upgrade-Insecure-Requests'] = '1'

#num_pages+1
for page in range(1,num_pages+1):
    url = f'https://www.amazon.co.uk/s?k={query}&page={page}&ref=nb_sb_noss'
    url = url.replace(" ", "%20")
#     # Parse the HTML content using BeautifulSoup
#     print(f'Page:{page}')
    print(url)
#     url = 'https://www.amazon.co.uk/s?k=boss+jewellery+for+men&crid=33F2J8F5ZMF98&sprefix=boss+jewellery+for+men%2Caps%2C208&ref=nb_sb_noss_1' 
    user_agent = random.choice(user_agent_list)
    opener = urllib.request.build_opener()
    opener.addheaders = [('User-agent', user_agent)]
    response = opener.open(url)
    html_content = response.read()
#     response = session.get(url)
#     html_content = response.content
    
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Extract the URLs of the product pages from the search results
    product_urls = []
    results = soup.find_all('div', {'class': 's-result-item'})
    for result in results:
        link = result.find('a', {'class': 'a-link-normal s-no-outline'})

        if link:
            u = 'https://www.amazon.co.uk'+link.get('href')
            product_urls.append(u)
    # List

    # Scrape the product details for each product page
    for product_url in product_urls[:-1]:
        url_key.append(product_url)
#         response = session.get(product_url)
#         html_content = response.content
        response = opener.open(product_url)
        html_content = response.read()
        soup = BeautifulSoup(html_content, 'html.parser')
        title = soup.find('span', attrs={'id': 'productTitle'}).get_text().strip()
        Title.append(title)
        if not soup.find('span', attrs={'class': 'a-price-whole'}):
            Price.append(' ')
        else:
            price1 = soup.find('span', attrs={'class': 'a-price-whole'}).get_text().strip()
            price2 = soup.find('span', attrs={'class': 'a-price-fraction'}).get_text().strip()
            Price.append(price1+price2)
        if not soup.find('span', attrs={'class': 'a-icon-alt'}):
            Rating.append(' ')
        else:
            rating = soup.find('span', attrs={'class': 'a-icon-alt'}).get_text().strip()
            Rating.append(rating)
        if not soup.find('div', attrs={'id': 'productDescription'}):
            Description.append(' ')
        else:
            description = soup.find('div', attrs={'id': 'productDescription'}).get_text().strip()
            Description.append(description)
        
        #Detail page
        specifications_container = soup.find("table", attrs={"id": "technicalSpecifications_section_1"})
        if specifications_container:
            specification_rows = specifications_container.find_all("tr")

            # Iterate through each specification row and extract the relevant information
            for row in specification_rows:
                # Extract the specification name
                name = row.find("th").text.strip()

                # Extract the specification value
                value = row.find("td").text.strip()

            # Print the extracted information
                df = df.append({'Name': name, 'Value': value, 'url': product_url}, ignore_index=True)
        else:
            df.append({'Name': ' ', 'Value': ' ', 'url': product_url}, ignore_index=True)
            
        print('---------------------------------------------------------')
        
        print(f'Title: {title}')
        print(f'Price: {price1+price2}')
        print(f'Rating: {rating}')
        print(f'Description: {description}')
        
 
        amz_scraper = AmazonScraper()
        
#         product_url = product_url.replace('dp', 'product-reviews')+'&reviewerType=all_reviews'
        for page_num in range(1, 100):
            if amz_scraper.scrapeReviews(url=product_url, page_num=page_num) ==[] or amz_scraper.scrapeReviews(url=product_url, page_num=page_num) ==None:
#                 reviews.extend(UserReview(' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '))
                break
            reviews.extend(amz_scraper.scrapeReviews(url=product_url, page_num=page_num))
            time.sleep(5)

#         Reviews = pd.DataFrame(reviews)
        if product_url == product_urls[-2]:
            print('\n')
            print('---------------------------------------------')
            print(f'page {page} is finished')
            print('---------------------------------------------')
            print('\n')
        
        
        time.sleep(5)

https://www.amazon.co.uk/s?k=tommy%20hilfiger%20jewellery%20men&page=1&ref=nb_sb_noss


---------------------------------------------------------
Title: Tommy Hilfiger Jewelry Men's Leather Bracelet Black - 2700534
Price: 28.03
Rating: 4.3 out of 5 stars
Description: Tommy Hilfiger Men's Braided Leather Bracelet with Stainless Steel Clasp, Black


---------------------------------------------------------
Title: Tommy Hilfiger Jewelry Men's Stainless Steel Pendant Necklace - 2790314
Price: 36.88
Rating: 4.6 out of 5 stars
Description: From sports-inspired classics to dressier styles, our jewelry pieces are united by timeless design and a sleek, stainless steel or leather finish.


---------------------------------------------------------
Title: Tommy Hilfiger Herren-Armband Edelstahl
Price: 39.53
Rating: 4.2 out of 5 stars
Description: 


---------------------------------------------------------
Title: Tommy Hilfiger Jewelry Men's Stainless Steel Chain Necklace - 2790365
Price: 56.00
Rating: 4.6 out of 5 stars
Description: Tommy Hilfiger Jewerly offers items for all occasions - all with a fashionable twist.From sports-inspired classics to dressier styles, our jewelry pieces are united by timeless design and a sleek, stainless steel or leather finish.


...

---------------------------------------------------------
Title: Tommy Hilfiger Jewelry Men's Stainless Steel Link Bracelet Stainless steel - 2790485
Price: 79.00
Rating: Previous page
Description: Tommy Hilfiger Jewerly offers items for all occasions - all with a fashionable twist. From sports-inspired classics to dressier styles, our jewelry pieces are united by timeless design and a sleek, stainless steel or leather finish.


---------------------------------------------------------
Title: Tommy Hilfiger Men's Link Bracelets
Price: 42.21
Rating: 4.2 out of 5 stars
Description: From sports-inspired classics to dressier styles, our jewelry pieces are united by timeless design and a sleek, stainless steel or leather finish.


---------------------------------------------------------
Title: Tommy Hilfiger Jewelry Women's Stainless Steel Bangle Bracelet Embellished with Crystals - 2780611
Price: 64.00
Rating: 4.8 out of 5 stars
Description: Tommy Hilfiger Jewerly offers items for all occasions - all with a fashionable twist.From sports-inspired classics to dressier styles, our jewelry pieces are united by timeless design and a sleek, stainless steel or leather finish.


---------------------------------------------------------
Title: Tommy Hilfiger Jewelry Men's Nylon Rope bracelet Navy Blue or Black
Price: 59.00
Rating: Previous page
Description: 




---------------------------------------------
page 3 is finished
---------------------------------------------




In [ ]:
for page in range(1,num_pages+1):
    url = f'https://www.amazon.co.uk/s?k={query}&page={page}&ref=nb_sb_noss'
    url = url.replace(" ", "%20")
#     # Parse the HTML content using BeautifulSoup
#     print(f'Page:{page}')
    print(url)
#     url = 'https://www.amazon.co.uk/s?k=boss+jewellery+for+men&crid=33F2J8F5ZMF98&sprefix=boss+jewellery+for+men%2Caps%2C208&ref=nb_sb_noss_1' 
    user_agent = random.choice(user_agent_list)
    opener = urllib.request.build_opener()
    opener.addheaders = [('User-agent', user_agent)]
    response = opener.open(url)
    html_content = response.read()
#     response = session.get(url)
#     html_content = response.content
    
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Extract the URLs of the product pages from the search results
    product_urls = []
    results = soup.find_all('div', {'class': 's-result-item'})
    for result in results:
        link = result.find('a', {'class': 'a-link-normal s-no-outline'})

        if link:
            u = 'https://www.amazon.co.uk'+link.get('href')
            product_urls.append(u)
    # List    
    for product_url in product_urls[:-1]:
        url_key.append(product_url)
#         response = session.get(product_url)
#         html_content = response.content
        response = opener.open(product_url)
        html_content = response.read()
        soup = BeautifulSoup(html_content, 'html.parser')
        title = soup.find('span', attrs={'id': 'productTitle'}).get_text().strip()
        Title.append(title)
        if not soup.find('span', attrs={'class': 'a-price-whole'}):
            Price.append(' ')
        else:
            price1 = soup.find('span', attrs={'class': 'a-price-whole'}).get_text().strip()
            price2 = soup.find('span', attrs={'class': 'a-price-fraction'}).get_text().strip()
            Price.append(price1+price2)
        if not soup.find('span', attrs={'class': 'a-icon-alt'}):
            Rating.append(' ')
        else:
            rating = soup.find('span', attrs={'class': 'a-icon-alt'}).get_text().strip()
            Rating.append(rating)
        if not soup.find('div', attrs={'id': 'productDescription'}):
            Description.append(' ')
        else:
            description = soup.find('div', attrs={'id': 'productDescription'}).get_text().strip()
            Description.append(description)
        
        #Detail page
        specifications_container = soup.find("table", attrs={"id": "technicalSpecifications_section_1"})
        if specifications_container:
            specification_rows = specifications_container.find_all("tr")

            # Iterate through each specification row and extract the relevant information
            for row in specification_rows:
                # Extract the specification name
                name = row.find("th").text.strip()

                # Extract the specification value
                value = row.find("td").text.strip()

            # Print the extracted information
                df = df.append({'Name': name, 'Value': value, 'url': product_url}, ignore_index=True)
        else:
            df.append({'Name': ' ', 'Value': ' ', 'url': product_url}, ignore_index=True)
            
        print('---------------------------------------------------------')
        
        print(f'Title: {title}')
        print(f'Price: {price1+price2}')
        print(f'Rating: {rating}')
        print(f'Description: {description}')
        
 
        amz_scraper = AmazonScraper()
#         product_url = product_url.replace('dp', 'product-reviews')+'&reviewerType=all_reviews'
        for page_num in range(1, 100):
            if amz_scraper.scrapeReviews(url=product_url, page_num=page_num) ==[] or amz_scraper.scrapeReviews(url=product_url, page_num=page_num) ==None:
#                 reviews.extend(UserReview(' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '))
                break
            reviews.extend(amz_scraper.scrapeReviews(url=product_url, page_num=page_num))
            time.sleep(5)

#         Reviews = pd.DataFrame(reviews)
        if product_url == product_urls[-2]:
            print('\n')
            print('---------------------------------------------')
            print(f'page {page} is finished')
            print('---------------------------------------------')
            print('\n')
        
        
        time.sleep(5)

NameError: name 'num_pages' is not defined

In [ ]:
data = {'Title': Title, 'Price': Price, 'Rating': Rating, 'Description': Description, 'Product_url': url_key}

# Create a DataFrame from the dictionary
product_describe = pd.DataFrame(data)
Reviews = pd.DataFrame(reviews)
df_sku = df[df['Name'] == ('Model number' or ' ')]
df_boss_women = Reviews.merge(df_sku, how='left', left_on='url', right_on='url')
df_boss_women = df_boss_women.merge(product_describe, how='left', left_on='url', right_on='Product_url')
df_boss_women_v1 = df_boss_women.dropna()
df_boss_women_v1 = df_boss_women_v1.drop('Product_url' ,axis=1)

In [ ]:
df_boss_women_v1.to_csv('boss_jew_women.csv', index=False)

In [43]:
data = {'Title': Title, 'Price': Price, 'Rating': Rating, 'Description': Description, 'Product_url': url_key}

# Create a DataFrame from the dictionary
product_describe = pd.DataFrame(data)
product_describe.head()

In [45]:
Reviews = pd.DataFrame(reviews)
Reviews.head()

In [47]:
Reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1222 entries, 0 to 1221
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   url                1222 non-null   object
 1   review_title       1222 non-null   object
 2   comment            1222 non-null   object
 3   rating             1222 non-null   object
 4   date               1222 non-null   object
 5   username           1222 non-null   object
 6   profile_url        1222 non-null   object
 7   verified_purchase  1222 non-null   bool  
dtypes: bool(1), object(7)
memory usage: 68.1+ KB


In [48]:
df.head()

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1107 entries, 0 to 1106
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Name    1107 non-null   object
 1   Value   1107 non-null   object
 2   url     1107 non-null   object
dtypes: object(3)
memory usage: 26.1+ KB


In [49]:
df_sku = df[df['Name'] == ('Model number' or ' ')]
df_sku.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 144 entries, 10 to 1106
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Name    144 non-null    object
 1   Value   144 non-null    object
 2   url     144 non-null    object
dtypes: object(3)
memory usage: 4.5+ KB


In [50]:
df_sku = df_sku[['Value','url']]
df_sku.head()

In [52]:
df_tommy_men = Reviews.merge(df_sku, how='left', left_on='url', right_on='url')
df_tommy_men = df_tommy_men.merge(product_describe, how='left', left_on='url', right_on='Product_url')
df_tommy_men_v1 = df_tommy_men.dropna()
df_tommy_men_v1 = df_tommy_men_v1.drop('Product_url' ,axis=1)

In [56]:
df_tommy_men_v1.to_csv('tommy_jew_men_v1.csv', index=False)

In [16]:
def amazonukscraper(query, user_agent_list = user_agent_list, num_pages = False):
    
    #find how many pages
    if not num_pages:
        page = 1
        url = f'https://www.amazon.co.uk/s?k={query}&page={page}&ref=nb_sb_noss'
        url = url.replace(" ", "%20")
        user_agent = random.choice(user_agent_list)
        opener = urllib.request.build_opener()
        opener.addheaders = [('User-agent', user_agent)]
        response = opener.open(url)
        html_content = response.read()
        soup = BeautifulSoup(html_content, 'html.parser')

        result_text = soup.find('div', {'class': 'a-section a-spacing-small a-spacing-top-small'}).get_text().strip()
        total_results = int(result_text.split()[2].replace(',', ''))

        results_per_page = int(result_text.split()[0].replace(',', '').split('-')[1])
        num_pages = total_results // results_per_page + (1 if total_results % results_per_page > 0 else 0)
        
    else:
        page = 1
        url = f'https://www.amazon.co.uk/s?k={query}&page={page}&ref=nb_sb_noss'
        url = url.replace(" ", "%20")
    
    #create df
    Title = []
    Price = []
    Rating = []
    Description = []
    url_key = []
    df = pd.DataFrame(columns=['Name', 'Value', 'url'])
    reviews = []
    
    #scrape for each page
    for page in range(1,num_pages+1):
        
        user_agent = random.choice(user_agent_list)
        opener = urllib.request.build_opener()
        opener.addheaders = [('User-agent', user_agent)]
        response = opener.open(url)
        html_content = response.read()

        soup = BeautifulSoup(html_content, 'html.parser')

        # Extract the URLs of the product pages from the search results
        product_urls = []
        results = soup.find_all('div', {'class': 's-result-item'})
        for result in results:
            link = result.find('a', {'class': 'a-link-normal s-no-outline'})

            if link:
                u = 'https://www.amazon.co.uk'+link.get('href')
                product_urls.append(u)

        # Scrape the product details for each product page
        for product_url in product_urls[:-1]:
            url_key.append(product_url)
            response = opener.open(product_url)
            html_content = response.read()
            soup = BeautifulSoup(html_content, 'html.parser')
            title = soup.find('span', attrs={'id': 'productTitle'}).get_text().strip()
            Title.append(title)
            if not soup.find('span', attrs={'class': 'a-price-whole'}):
                Price.append(' ')
            else:
                price1 = soup.find('span', attrs={'class': 'a-price-whole'}).get_text().strip()
                price2 = soup.find('span', attrs={'class': 'a-price-fraction'}).get_text().strip()
                if float(price1+price2) > 1000:
                    Price.append(0)
                else:
                    Price.append(price1+price2)
            if not soup.find('span', attrs={'class': 'a-icon-alt'}):
                Rating.append(' ')
            else:
                rating = soup.find('span', attrs={'class': 'a-icon-alt'}).get_text().strip()
                Rating.append(rating)
            if not soup.find('div', attrs={'id': 'productDescription'}):
                Description.append(' ')
            else:
                description = soup.find('div', attrs={'id': 'productDescription'}).get_text().strip()
                Description.append(description)

            #Detail page
            specifications_container = soup.find("table", attrs={"id": "technicalSpecifications_section_1"})
            if specifications_container:
                specification_rows = specifications_container.find_all("tr")

                # Iterate through each specification row and extract the relevant information
                for row in specification_rows:
                    # Extract the specification name
                    name = row.find("th").text.strip()

                    # Extract the specification value
                    value = row.find("td").text.strip()

                # Print the extracted information
                    df = df.append({'Name': name, 'Value': value, 'url': product_url}, ignore_index=True)
            else:
                df.append({'Name': ' ', 'Value': ' ', 'url': product_url}, ignore_index=True)

            print('---------------------------------------------------------')

            print(f'Title: {title}')
            print(f'Price: {price1+price2}')
            print(f'Rating: {rating}')
            print(f'Description: {description}')

            #Scraping comments
            amz_scraper = AmazonScraper()
            for page_num in range(1, 100):
                if amz_scraper.scrapeReviews(url=product_url, page_num=page_num) ==[] \
                or amz_scraper.scrapeReviews(url=product_url, page_num=page_num) ==None:
                    break
                reviews.extend(amz_scraper.scrapeReviews(url=product_url, page_num=page_num))
                time.sleep(5)

            if product_url == product_urls[-2]:
                print('\n')
                print('---------------------------------------------')
                print(f'page {page} is finished')
                print('---------------------------------------------')
                print('\n')


            time.sleep(5)
            
    

    # Create a DataFrame from the dictionary
    data = {'Title': Title, 'Price': Price, 'Rating': Rating, 'Description': Description, 'Product_url': url_key}
    product_describe = pd.DataFrame(data)
    Reviews = pd.DataFrame(reviews)
    df_sku = df[df['Name'] == ('Model number' or ' ')]
    df_merge = Reviews.merge(df_sku, how='left', left_on='url', right_on='url')
    df_merge = df_merge.merge(product_describe, how='left', left_on='url', right_on='Product_url')
    df_merge_v1 = df_merge.dropna()
    df_merge_v1 = df_merge_v1.drop('Product_url' ,axis=1)
    
#     #export csv
#     df_merge_v1.to_csv(file_name, index=False)
    print('---------------------------------------------')
    print('---------------------------------------------')
    print('All done')
    
    return df_merge_v1

In [22]:
df = amazonukscraper('boss jewellery women', num_pages=1)

---------------------------------------------------------
Title: BOSS Jewelry Women's Medallion Collection Hoop Earrings - 1580231
Price: 44.00
Rating: 4.5 out of 5 stars
Description: BOSS jewellery reflects the sophisticated character and craftsmanship for which all HUGO BOSS products are known. Synonymous with fashion, it stands out in the world of quality branded jewellerywith a style and presence of its own.


---------------------------------------------------------
Title: BOSS Jewelry Women's Beads Collection Chain Bracelet - 1580075
Price: 53.00
Rating: 4.4 out of 5 stars
Description: BOSS jewellery reflects the sophisticated character and craftsmanship for which all HUGO BOSS products are known. Synonymous with fashion, it stands out in the world of quality branded jewellerywith a style and presence of its own.


---------------------------------------------------------
Title: BOSS Jewelry Women's Medallion Collection Chain Bracelet - 1580227
Price: 50.15
Rating: 4.5 out of 5 stars
Description: BOSS jewellery reflects the sophisticated character and craftsmanship for which all HUGO BOSS products are known. Synonymous with fashion, it stands out in the world of quality branded jewellerywith a style and presence of its own.


---------------------------------------------------------
Title: BOSS Jewelry Women's Tessa Collection Necklace - 1580200
Price: 97.00
Rating: 4.7 out of 5 stars
Description: BOSS jewellery reflects the sophisticated character and craftsmanship for which all HUGO BOSS products are known. Synonymous with fashion, it stands out in the world of quality branded jewellerywith a style and presence of its own.


---------------------------------------------------------
Title: BOSS Jewelry Women's CALY Collection Chain Bracelet in Stainless steel
Price: 80.00
Rating: Previous page
Description: 


...

---------------------------------------------------------
Title: BOSS Jewelry Men's Orlado Collection Chain Bracelet - 1580359M
Price: 62.60
Rating: 3.1 out of 5 stars
Description: Dress like a BOSS. BOSS jewelry reflects the dressing excellence codes and design for which all HUGO BOSS products are known. Synonymous with fashion, it stands out in the world of quality branded jewelry with a style and presence of its own. Tailored for the man and woman who draw attention in all situations. #BeYourOwnBOSS


---------------------------------------------------------
Title: BOSS Jewelry Women's LYSSA Collection Hoop Earrings Yellow gold - 1580346
Price: 63.00
Rating: Previous page
Description: Dress like a BOSS. BOSS jewelry reflects the dressing excellence codes and design for which all HUGO BOSS products are known. Synonymous with fashion, it stands out in the world of quality branded jewelry with a style and presence of its own. Tailored for the man and woman who draw attention in all situations. #BeYourOwnBOSS.


---------------------------------------------------------
Title: BOSS Jewelry Men's Lander Collection Pendant Necklace - 1580180
Price: 69.00
Rating: 4.0 out of 5 stars
Description: BOSS jewellery reflects the sophisticated character and craftsmanship for which all HUGO BOSS products are known. Synonymous with fashion, it stands out in the world of quality branded jewellerywith a style and presence of its own.




---------------------------------------------
page 1 is finished
---------------------------------------------




---------------------------------------------
---------------------------------------------
All done


In [20]:
df.to_csv('XXX.csv', index=False)